In [1]:
import arimax as model
import pandas as pd
import numpy as np
import statistics as s

In [2]:
df=pd.read_csv("Data1997-2022.csv")
df['Intercept'] = np.ones(len(df))
df

,Unnamed: 0,CPI,Monthly salary,Policy rate,Unemployed,Intercept
0,0,69.3,20126,3.50,4.0,1.0
1,1,69.6,20126,3.50,3.7,1.0
2,2,69.7,20126,3.50,3.6,1.0
3,3,69.7,20126,3.50,3.4,1.0
4,4,69.8,20126,3.50,3.2,1.0
...,...,...,...,...,...,...
307,307,123.9,53150,1.75,1.6,1.0
308,308,125.6,53150,1.75,1.6,1.0
309,309,126.0,53150,2.25,1.6,1.0
310,310,125.8,53150,2.50,1.6,1.0


In [3]:
cpi_diff12 = df['CPI'].diff(12)
exog_data = df[['Intercept', 'Monthly salary', 'Policy rate', 'Unemployed']]

cpi_diff12


0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
307    7.6
308    8.1
309    8.8
310    7.7
311    7.0
Name: CPI, Length: 312, dtype: float64

In [8]:
ARX = model.ARMAX(2, 0, 3, cpi_diff12[12:], exog_data[12:])
evo_init = np.array([-1.6242, -0.8331])
var_init = 72
beta_init = np.array([1.101e-08, -0.0001, 0.0251, 0.0449])
x_0, P_0 = np.ones(ARX.p)*s.mean(cpi_diff12[12:]), np.zeros((ARX.p,ARX.p))
np.fill_diagonal(P_0, s.variance(cpi_diff12[12:]))
P_0 = P_0.reshape((1,ARX.p*ARX.p))
opt = ARX.fit_kalman(evo_init, var_init, beta_init, x_0, P_0)

2302.4634840640933
2302.4634762267647
2302.463485044833
2302.4634841598436
2302.463483751723
2302.450888217358
2302.4634833845193
2302.463483197403
2803635323.105889
2803635030.9709697
2803635159.810574
2803635321.9451127
2803635323.107959
2803635405.8459535
2803635323.1102843
2803635323.111778
2237.723222664978
2237.723221263898
2237.7232273288414
2237.723222787531
2237.7232226684714
2237.7232180506007
2237.7232226597657
2237.723222695692
15380.696600856674
15380.696806928228
15380.695652194228
15380.69660098027
15380.696600856774
15380.696603070672
15380.696600857278
15380.696600856912
2237.0980169288578
2237.098015534179
2237.098021626353
2237.0980170514053
2237.0980169328695
2237.098032953743
2237.0980169246914
2237.0980169611153
2236.477398858562
2236.477397463729
2236.4774035875457
2236.4773989811056
2236.4773988630955
2236.4774354671313
2236.4773988554484
2236.477398892363
4052.451659309868
4052.451607349752
4052.450723576479
4052.451659433475
4052.451659310255
4052.451666679917

In [9]:
opt.x

array([-1.76627211e+00, -9.74057221e-01, -1.56318981e+01, -2.08759968e+00,
        1.71321369e-04, -1.12283968e+00,  1.44248221e-01])

In [6]:
ARX.summary()

                                    Results                                     
Dep. Variable:                                  CPI
Model:                              ARIMAX( 2 , 0 )
No. Observations:                               300
AIC                                        AICvalue
BIC                                        BICvalue
Log Likelihood                             Logvalue
                     coef    std.err          z      P>|z|     [0.025     0.975]
--------------------------------------------------------------------------------
Intercept       4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
Monthly salary  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
Policy rate     4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
Unemployed      4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
ar.L1           4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00  4.236e+00
ar.L2           4.236e+00  4.236e+00  4.